## Chain-of-Thought Prompting

In [3]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.environ['OPENAI_API_KEY']

def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products: 

Brand: Daikin
Category: Split Air Conditioner
Model Number: DTKL50TV16U
Warranty: 1 year comprehensive, 5 years on compressor
Rating: 4.5 stars
Features: Inverter technology, energy-efficient cooling, 3D airflow, PM2.5 filter, sleep mode, auto-restart function
Description: The Daikin DTKL50TV16U is a powerful split air conditioner that provides efficient cooling with its inverter technology. It ensures clean and fresh air with its PM2.5 filter, and the 3D airflow evenly distributes cool air in the room. It comes with a sleep mode and auto-restart function for added convenience.
Price: $650

Brand: LG
Category: Window Air Conditioner
Model Number: LW8016ER
Warranty: 1 year parts and labor, 5 years on the sealed system
Rating: 4.3 stars
Features: Remote control, energy-saving mode, multiple cooling modes, 24-hour programmable timer, washable filter
Description: The LG LW8016ER is a reliable window air conditioner that offers efficient cooling for smaller rooms. It comes with a remote control for convenient operation and has various cooling modes to suit different preferences. The energy-saving mode helps reduce power consumption, and the programmable timer allows you to set the desired cooling schedule.
Price: $299

Brand: Carrier
Category: Ductless Mini-Split Air Conditioner
Model Number: 38MGQC183
Warranty: 5 years parts, 7 years on the compressor
Rating: 4.8 stars
Features: Wi-Fi control, variable-speed operation, dehumidification mode, sleep mode, intelligent auto restart, noise reduction technology
Description: The Carrier 38MGQC183 is a ductless mini-split air conditioner that offers flexible cooling solutions. With its Wi-Fi control, you can conveniently operate the AC using your smartphone. The variable-speed operation ensures energy efficiency, and the dehumidification mode helps maintain a comfortable humidity level. It also features intelligent auto restart and noise reduction technology for a seamless and quiet operation.
Price: $1,200

Brand: Mitsubishi Electric
Category: Ceiling Cassette Air Conditioner
Model Number: MLZ-KP09NA
Warranty: 5 years parts, 7 years on the compressor
Rating: 4.6 stars
Features: 360-degree airflow, multiple fan speeds, auto vane control, anti-allergy enzyme filter, quiet operation, wireless remote controller
Description: The Mitsubishi Electric MLZ-KP09NA is a ceiling cassette air conditioner suitable for commercial spaces. It offers 360-degree airflow to evenly cool the entire room. The anti-allergy enzyme filter helps purify the air, and the quiet operation ensures a comfortable environment. The wireless remote controller provides easy access to the AC's settings and features.
Price: $1,800

Brand: Panasonic
Category: Portable Air Conditioner
Model Number: CW-XC145HU
Warranty: 1 year parts and labor, 5 years on the compressor
Rating: 4.2 stars
Features: Self-evaporating system, 3 operating modes, 24-hour programmable timer, adjustable louvers, remote control, built-in air purifier
Description: The Panasonic CW-XC145HU is a portable air

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that AC X is bigger than \
AC  Y, or that AC Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.

"""

user_message = f"""
by how much is the Mitsubishi AC more expensive \
than the Daikin AC"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

user_message = f"""
do you sell tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try again later."
    
print(final_response)

Step 1:#### The user is asking about the price difference between two specific products.
Step 2:#### The two products in question are the Mitsubishi Electric MLZ-KP09NA and the Daikin DTKL50TV16U.
Step 3:#### The user assumes that the Mitsubishi AC is more expensive than the Daikin AC.
Step 4:#### The assumption is true. The Mitsubishi Electric MLZ-KP09NA is priced at $1,800, while the Daikin DTKL50TV16U is priced at $650.
Response to user:#### The Mitsubishi Electric MLZ-KP09NA is priced at $1,800, which is significantly more expensive than the Daikin DTKL50TV16U, which is priced at $650.
Step 1:#### The user is asking if the store sells TVs.
Step 2:#### TVs are not listed in the available products.
Response to user:#### I'm sorry, but we do not sell TVs. Our store specializes in air conditioning products.
I'm sorry, but we do not sell TVs. Our store specializes in air conditioning products.
